In [13]:
import pandas as pd
import boto3
import os

In [8]:
s3 = boto3.client('s3')

In [9]:
bucket_name = 'datainternetaccess-cleaning'

In [8]:
# Cargar el archivo Excel
file_path = 'data/articles-383732_archivo_xls.xlsx'
xls = pd.ExcelFile(file_path)

In [10]:
# Crear el bucket
try:
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={
        'LocationConstraint': 'us-west-1'  # Ajusta la región según tus necesidades
    })
    print(f"Bucket {bucket_name} creado con éxito.")
except Exception as e:
    print(f"Error al crear el bucket: {e}")


Bucket datainternetaccess-cleaning creado con éxito.


In [11]:
# Subir los archivos de la carpeta 'data'
folder_path = 'data/'

In [14]:
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)

    if os.path.isfile(file_path):
        try:
            s3.upload_file(file_path, bucket_name, file_name)
            print(f"Archivo {file_name} subido con éxito al bucket {bucket_name}.")
        except Exception as e:
            print(f"Error al subir el archivo {file_name}: {e}")

Archivo LÍNEAS_EN_SERVICIO_POR_ESTRATO_Y_MUNICIPIO_DE_TELEFONÍA_PÚBLICA_BÁSICA_CONMUTADA_19.csv subido con éxito al bucket datainternetaccess-cleaning.
Archivo articles-383732_archivo_xls.xlsx:Zone.Identifier subido con éxito al bucket datainternetaccess-cleaning.
Archivo INFORMACIÓN_TRIMESTRAL_DE_TRÁFICO_DE_INTERNET_MÓVIL_SUSCRIPCIÓN,_POR_PROVEEDOR_14.csv subido con éxito al bucket datainternetaccess-cleaning.
Archivo INFORMACIÓN_TRIMESTRAL_DE_COBERTURA_MUNICIPAL_DEL_SERVICIO_MÓVIL_18.csv subido con éxito al bucket datainternetaccess-cleaning.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_6.csv subido con éxito al bucket datainternetaccess-cleaning.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_DEPARTAMENTO,_MUNICIPIO_Y_POBLACIÓN_3.csv subido con éxito al bucket datainternetaccess-cleaning.
Archivo INFORMACIÓN_TRIMESTRAL_ABONADOS_DE_TELEFONÍA_MÓVIL_15.csv subido con éxit

In [15]:
# Directorio local donde guardar los archivos descargados
download_folder = 'data/'

In [16]:
# Verificar si los archivos están en S3 y descargarlos si no están
for file_name in os.listdir(download_folder):
    local_file_path = os.path.join(download_folder, file_name)

    try:
        # Verificar si el archivo existe en el bucket
        s3.head_object(Bucket=bucket_name, Key=file_name)
        print(f"El archivo {file_name} ya existe en el bucket {bucket_name}.")
    except Exception as e:
        print(f"El archivo {file_name} no existe en el bucket. Descargando...")

        try:
            # Descargar el archivo desde S3
            s3.download_file(bucket_name, file_name, local_file_path)
            print(f"Archivo {file_name} descargado con éxito.")
        except Exception as download_error:
            print(f"Error al descargar el archivo {file_name}: {download_error}")

El archivo LÍNEAS_EN_SERVICIO_POR_ESTRATO_Y_MUNICIPIO_DE_TELEFONÍA_PÚBLICA_BÁSICA_CONMUTADA_19.csv ya existe en el bucket datainternetaccess-cleaning.
El archivo articles-383732_archivo_xls.xlsx:Zone.Identifier ya existe en el bucket datainternetaccess-cleaning.
El archivo INFORMACIÓN_TRIMESTRAL_DE_TRÁFICO_DE_INTERNET_MÓVIL_SUSCRIPCIÓN,_POR_PROVEEDOR_14.csv ya existe en el bucket datainternetaccess-cleaning.
El archivo INFORMACIÓN_TRIMESTRAL_DE_COBERTURA_MUNICIPAL_DEL_SERVICIO_MÓVIL_18.csv ya existe en el bucket datainternetaccess-cleaning.
El archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_6.csv ya existe en el bucket datainternetaccess-cleaning.
El archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_DEPARTAMENTO,_MUNICIPIO_Y_POBLACIÓN_3.csv ya existe en el bucket datainternetaccess-cleaning.
El archivo INFORMACIÓN_TRIMESTRAL_ABONADOS_DE_TELEFONÍA_MÓVIL_15.csv ya existe en el bu

In [9]:
# Obtener la lista de hojas, excluyendo las que no necesitas
sheet_names = [sheet for sheet in xls.sheet_names if sheet not in ['PORTADA', 'CONTENIDO', '18']]

In [10]:
sheet_names

['1',
 '2',
 '3',
 '4,1',
 '4,2',
 '4,3',
 '4,4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17']

In [11]:
# Función para limpiar y exportar datos de cada hoja
def export_sheet_to_csv(sheet_name, xls, index):
    # Leer la hoja, comenzando desde la fila 6 (excluyendo las 5 primeras)
    df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=5)

    # Obtener el nombre del archivo desde la fila 1 (segunda fila en el índice 0), columna A (índice 0)
    try:
        sheet_name_row = pd.read_excel(xls, sheet_name=sheet_name, nrows=2).iloc[0, 0]  # Segunda fila, índice 0
    except IndexError:
        sheet_name_row = f"archivo_{sheet_name}"  # Nombre predeterminado en caso de error

    # Crear el nombre del archivo utilizando el contenido de la segunda fila y agregando un sufijo único
    clean_file_name = f"{sheet_name_row}_{index}.csv".replace(" ", "_").replace("/", "_")

    # Guardar el CSV con el nombre generado
    df.to_csv(clean_file_name, index=False)
    print(f"Archivo {clean_file_name} creado con éxito.")

In [12]:
# Exportar cada hoja a un archivo CSV
for index, sheet in enumerate(sheet_names, start=1):
    export_sheet_to_csv(sheet, xls, index)

print("Exportación completa.")

Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR__1.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_DEPARTAMENTO_Y_POBLACIÓN_2.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_DEPARTAMENTO,_MUNICIPIO_Y_POBLACIÓN_3.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_4.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_5.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_6.csv creado con éxito.
Archivo INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_7.csv creado co

In [18]:
import os
import pandas as pd

# Ruta de la carpeta donde están los CSVs
folder_path = '../Limpieza'

# Función para leer las columnas de cada archivo CSV
def listar_columnas_csv(folder_path):
    archivos_csv = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    columnas_por_archivo = {}

    for archivo in archivos_csv:
        archivo_path = os.path.join(folder_path, archivo)
        try:
            df = pd.read_csv(archivo_path, nrows=5)  # Leer solo las primeras filas para obtener las columnas
            columnas_por_archivo[archivo] = df.columns.tolist()
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")

    return columnas_por_archivo

# Ejecutar la función y obtener las columnas de cada archivo CSV
columnas_por_archivo = listar_columnas_csv(folder_path)

# Mostrar el resultado
for archivo, columnas in columnas_por_archivo.items():
    print(f"Archivo: {archivo}")
    print(f"Columnas: {columnas}\n")


Archivo: LÍNEAS_EN_SERVICIO_POR_ESTRATO_Y_MUNICIPIO_DE_TELEFONÍA_PÚBLICA_BÁSICA_CONMUTADA_19.csv
Columnas: ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO']

Archivo: INFORMACIÓN_TRIMESTRAL_DE_TRÁFICO_DE_INTERNET_MÓVIL_SUSCRIPCIÓN,_POR_PROVEEDOR_14.csv
Columnas: ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)']

Archivo: INFORMACIÓN_TRIMESTRAL_DE_COBERTURA_MUNICIPAL_DEL_SERVICIO_MÓVIL_18.csv
Columnas: ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE.1', 'MUNICIPIO', 'CABECERA MUNICIPAL ', 'CÓDIGO DANE.2', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+, HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G']

Archivo: INFORMACIÓN_TRIMESTRAL_DE_ACCESOS_FIJOS_A_INTERNET_POR_PROVEEDOR,_DEPARTAMENTO,_MUNICIPIO,_SEGMENTO,_TECNOLOGIA,_Y_VELOCIDAD_DE_CONEXIÓN_6.csv
Columnas: ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDI

In [2]:
!pip install boto3

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import boto3

ImportError: cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (/home/hadoop/data-projects/InternetAccessColombia/.venv/lib/python3.10/site-packages/urllib3/util/ssl_.py)